In [1]:
!pip install transformers datasets accelerate huggingface_hub

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [2]:
import os
import json
from getpass import getpass
from huggingface_hub import login

In [3]:
token = getpass(prompt="enter huggingface token")
login(token=token)

enter huggingface token ········


In [4]:
dir_name = "aaron_data"
   
if os.path.isfile(dir_name + '/topics.json'):
    with open(dir_name + '/topics.json') as t:
        topics = json.load(t)

    topics = topics.keys()
    tfiles = [dir_name + f"""/{"-".join(t.lower().split(' '))}.json""" for t in topics]
    tfiles = [tf for tf in tfiles if os.path.isfile(tf)]
    
    data = []
    for tf in tfiles:
        with open(tf) as t:
            data.extend(json.load(t))

    print(f'there are {len(data)} rows in the dataset')

there are 180 rows in the dataset


In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

In [6]:
dataset = load_dataset("json", data_files=tfiles)

In [7]:
def preprocess(row):
    row['input'] = f'<question>\n{row["question"]}\n</question>\n\n<documents>\n{row["documents"]}\n</documents>\n\n<answer>'
    return row

In [8]:
dataset = dataset.map(preprocess).shuffle(seed=42)

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

In [9]:
print(dataset['train'][0]['input'])

<question>
What techniques are most effective for urban gardening in small spaces?
</question>

<documents>
Document 1: Coffee varietals, distinct genetic variants within species like Coffea arabica and Coffea canephora (robusta), significantly influence flavor profiles and growing characteristics. Traditional arabica varietals include Typica (noted for sweet, clean flavors but low yield) and Bourbon (known for complex acidity and moderate body). Hybridization has produced numerous cultivars addressing specific agricultural challenges while maintaining quality attributes. Examples include Caturra, a natural Bourbon mutation with compact growth allowing denser planting; Catuai, combining Mundo Novo's vigor with Caturra's size; and Gesha (or Geisha), an Ethiopian-origin varietal that gained fame after winning the 2004 Panama Cup of Excellence with its distinctive floral and tea-like characteristics. Modern breeding programs increasingly focus on developing varieties combining disease res

In [10]:
model_id = "meta-llama/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [11]:
model = model.to(torch.device('cpu'))

In [12]:
tokenizer.pad_token = tokenizer.eos_token

In [13]:
def tokenize_function(row):
    prompt = row["input"]
    full_text = prompt + "\n" + f'row["answer"]\n</answer>'
    
    tokenized = tokenizer(
        full_text,
        truncation=True,
        padding="max_length",
    )
    
    prompt_tokenized = tokenizer(
        prompt,
        truncation=True,
        add_special_tokens=False  # Ensure no extra tokens are added
    )
    prompt_length = len(prompt_tokenized["input_ids"])
    
    # Copy the tokenized full text to create labels and mask out prompt tokens
    labels = tokenized["input_ids"].copy()
    labels[:prompt_length] = [-100] * prompt_length  # -100 tells PyTorch to ignore these tokens in loss computation
    
    tokenized["labels"] = labels
    return tokenized

In [14]:
tokenized_dataset = dataset.map(tokenize_function)

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

In [15]:
from peft import LoraConfig, get_peft_model

# Define LoRA Configuration
lora_config = LoraConfig(
    r=4,               # Low-rank dimension
    lora_alpha=32,     # Scaling factor
    lora_dropout=0.1,  # Dropout for regularization
    bias="none",       # No bias training
    task_type="CAUSAL_LM"  # Task type: Causal Language Modeling
)

# Wrap the model with LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 425,984 || all params: 1,236,240,384 || trainable%: 0.0345


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./llama-1b-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    use_cpu=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],  # or the appropriate split name
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


[2025-03-06 02:30:46,629] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/opt/rh/gcc-toolset-12/root/usr/libexec/gcc/x86_64-redhat-linux/12/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status
